<a href="https://colab.research.google.com/github/jlembury/kickflip-cartography-demo/blob/main/kickflip_cartography_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="margin-left: 0; margin-right: auto;">
    <tr>
        <td><img src="https://github.com/jlembury/kickflip-cartography-demo/blob/main/img/img-nada-kJvjDlqTkR0-unsplash.jpg?raw=1"></td>
        <td><h1 style="margin-bottom: 0;">Kickflip Cartography: </h1>
            <h3 style="margin-top: 0;">Finding the Perfect Location for a Skate Park</h3><br><br>
            <h4 style="margin-bottom: 0;">GIS Awareness Week 2025, hosted by AIRC x Ecologik</h4>
            <h4 style="margin-top: 0; margin-bottom: 0;">Jessica Embury <em>(she/her/hers)</em></h4>
            <h4 style="margin-top: 0; margin-bottom: 0;">Email: <a href="mailto:jessica@gisjess.com">jessica@gisjess.com</a></h4>
        </td>
    </tr>
</table>
<hr>
<br>

### Project Overview
A local community group wants to build a skate park somewhere in the City of San Diego. The group wants to create a safe hangout for skateboarders <em>and</em> maximize access to skate parks throughout the city. To meet these objectives, the site for the future skate park must:
1. be located in a <b>San Diego public park</b> at least 1 mile from an <b>existing skate park</b>, and
2. be located in a <b>zip code</b> with demonstrated need, per <b>2025 San Diego police call records</b>.
<br>
<hr>
<br>

### Data Sources
- <em>parks</em>: San Diego Regional Data Warehouse, https://tinyurl.com/sd-data-warehouse
- <em>skateParks</em>: The City of San Diego, https://www.sandiego.gov/park-and-recreation/centers/skateparks
- <em>zipcodes</em>: San Diego Regional Data Warehouse, https://tinyurl.com/sd-data-warehouse
- <em>policeCalls</em>: Data SD, https://data.sandiego.gov/datasets/police-calls-for-service/
<br>
<hr>
<br>

### Spatial Operations
<img src="https://github.com/jlembury/kickflip-cartography-demo/blob/main/img/img-spatial-operations.jpg?raw=1">
<hr>

###Demo Set Up

In [ ]:
# Install libraries
!pip install geopandas

In [ ]:
# Import libraries
import pandas as pd
import geopandas as gpd
import time
import numpy as np
from geopy.geocoders import Nominatim
from shapely.geometry import Point

In [ ]:
# Assign input data URLs to variables
skateParksFile = "https://raw.githubusercontent.com/jlembury/kickflip-cartography-demo/refs/heads/main/data/skate-park-locations.csv"
parksFile = "https://raw.githubusercontent.com/jlembury/kickflip-cartography-demo/refs/heads/main/data/Parks_SD.geojson"
policeCallsFile = "https://raw.githubusercontent.com/jlembury/kickflip-cartography-demo/refs/heads/main/data/skateboarding-police-calls-2025.csv"
zipcodesFile = "https://raw.githubusercontent.com/jlembury/kickflip-cartography-demo/refs/heads/main/data/Zipcodes.geojson"

<hr>

###Objective 1
Identify San Diego parks that are more than 1 mile from a skate park.

Step 1. <u>Geocode</u> the street addresses in <em>skateParks</em> → <em>skateParkPoints</em>

In [ ]:
# Load skateParks data
skateParks = pd.read_csv(skateParksFile)
print(f"There are {len(skateParks)} skate parks in San Diego.")
skateParks.head(2)

There are 13 skate parks in San Diego.


,skate_park_name,address
0,Bill and Maxine Wilson Skate Park,"702 S 30th St, San Diego, CA 92113"
1,Carmel Valley Skate Park,"12600 El Camino Real, San Diego, CA 92130"


In [ ]:
# Create geolocator to interact with the geocoding service, Nominatim.
geolocator = Nominatim(user_agent="kickflip-cartography-demo")

In [ ]:
# Geocode the address text to get latitude and longitude coordinates.
geometry = []  # Empty list object to store mappable point geometry
for i,row in skateParks.iterrows():  # For each skate park
  location = geolocator.geocode(skateParks.at[i, "address"]) # Geocode the address text
  geometry.append(Point(location.longitude, location.latitude))  # Create a point and store it in the geometry list
  time.sleep(1)  # Wait 1 second, per terms of service

# Merge skate park data and skate park geometry
skateParkPoints = gpd.GeoDataFrame(skateParks, geometry=geometry, crs="EPSG:4326")
skateParkPoints.to_crs("EPSG:3857", inplace=True)  # select projection
skateParkPoints.head(2)

,skate_park_name,address,geometry
0,Bill and Maxine Wilson Skate Park,"702 S 30th St, San Diego, CA 92113",POINT (-13038806.739 3855340.433)
1,Carmel Valley Skate Park,"12600 El Camino Real, San Diego, CA 92130",POINT (-13050851.474 3885962.986)


Step 2. Use a <u>buffer</u> operation to create 1-mile zones around the points in <em>skateParkPoints</em>  → <em>skateParkBuffers</em>

In [ ]:
# Prepare data for buffers
skateParkBuffers = skateParkPoints[["skate_park_name", "address"]]

# create geometry for 1-mile buffers (1,610 meters)
skateParkBuffers["geometry"] = skateParkPoints.geometry.buffer(1610)
skateParkBuffers = skateParkBuffers.set_geometry("geometry")
skateParkBuffers.head(2)

,skate_park_name,address,geometry
0,Bill and Maxine Wilson Skate Park,"702 S 30th St, San Diego, CA 92113","POLYGON ((-13037196.739 3855340.433, -13037204..."
1,Carmel Valley Skate Park,"12600 El Camino Real, San Diego, CA 92130","POLYGON ((-13049241.474 3885962.986, -13049249..."


Step 3. Use a <u>difference</u> operation to remove <em>parks</em> that overlap with <em>skateParkBuffers</em> → <em>parkCandidates</em>

In [ ]:
# Load parks data
parks = gpd.read_file(parksFile)[["full_name", "address_lo", "community", "geometry"]].rename(columns={"full_name":"park_name", "address_lo":"address"})
parks.to_crs("EPSG:3857", inplace=True)  # set projection

print(f"There are {len(parks)} parks in San Diego.")
parks.head(2)


There are 301 parks in San Diego.


,park_name,address,community,geometry
0,Oak Neighborhood Park,"5235 Maple St. , 92105",MID-CITY: EASTERN AREA,"POLYGON ((-13033555.404 3859934.792, -13033555..."
1,San Carlos Community Park & Recreation Center,"6445 Lake Badin Ave., 92119",NAVAJO,"POLYGON ((-13026633.271 3868746.992, -13026656..."


In [ ]:
# Use a spatial join to merge data when a park is within a 1-mile skate park buffer.
parkCandidates = parks.sjoin(skateParkBuffers, how="left")

# Save parks data for parks that did NOT join with a skate park (more than 1 mile away)
parkCandidates = parkCandidates[parkCandidates['skate_park_name'].isna()][["park_name", "address_left", "community", "geometry"]].rename(columns={"address_left":"address"})

print(f"There are {len(parkCandidates)} San Diego parks that are more than 1 mile from a skate park.")
parkCandidates.head(2)

There are 246 San Diego parks that are more than 1 mile from a skate park.


,park_name,address,community,geometry
0,Oak Neighborhood Park,"5235 Maple St. , 92105",MID-CITY: EASTERN AREA,"POLYGON ((-13033555.404 3859934.792, -13033555..."
1,San Carlos Community Park & Recreation Center,"6445 Lake Badin Ave., 92119",NAVAJO,"POLYGON ((-13026633.271 3868746.992, -13026656..."


<hr>

###Objective 2
Find park(s) located in the zipcode with the most skateboarding police calls.

Step 4. <u>Geocode</u> the skateboarding incident locations from <em>policeCalls</em> → <em>policeCallPoints</em>**bold text**

In [ ]:
# Load policeCalls data.
policeCalls = pd.read_csv(policeCallsFile)
print(f"In 2025, there were {len(policeCalls)} police incidents about skateboarding.")
policeCalls.head(2)

In 2025, there were 13 police incidents about skateboarding.


,incident-number,address
0,E25040018295,"2400 4th Ave, San Diego, CA"
1,E25060021798,"3100 University Ave, San Diego, CA"


In [ ]:
# Geocode the address text to get latitude and longitude coordinates.
geometry = [] # Empty list object to store mappable point geometry
for i,row in policeCalls.iterrows():  # For each incident
  location = geolocator.geocode(policeCalls.at[i, "address"])  # Geocode the address text
  geometry.append(Point(location.longitude, location.latitude))  # Create a point and store it in the geometry list
  time.sleep(1)  # Wait 1 second, per terms of service

# Merge police call data and police call geometry
policeCallPoints = gpd.GeoDataFrame(policeCalls, geometry=geometry, crs="EPSG:4326")
policeCallPoints.to_crs("EPSG:3857", inplace=True)  # select projection
policeCallPoints.head(2)

,incident-number,address,geometry
0,E25040018295,"2400 4th Ave, San Diego, CA",POINT (-13042335.7 3859582.767)
1,E25060021798,"3100 University Ave, San Diego, CA",POINT (-13038495.601 3861975.731)


Step 5. Use a <u>within</u> operation to count the number of incidents per zipcode in <em>zipcodes</em> → <em>zipcodeCounts</em>

In [ ]:
# Load zipcodes data.
zipcodes = gpd.read_file(zipcodesFile).query("COMMUNITY == 'San Diego'")[["ZIP", "geometry"]].rename(columns={"ZIP":"zipcode"})
zipcodes.to_crs("EPSG:3857", inplace=True)  # select projection.
print(f"There are {len(zipcodes)} zipcodes in San Diego.")
zipcodes.head(2)

There are 36 zipcodes in San Diego.


,zipcode,geometry
2,92106,"POLYGON ((-13048704.485 3862395.903, -13048839..."
3,92154,"POLYGON ((-13035373.118 3843521.696, -13035369..."


In [ ]:
# Use a spatial join to merge data when a police incident occurred within a zipcode.
zipcodeCounts = zipcodes.sjoin(policeCallPoints, how="left")

# Find the zipcode with the highest number of police incidents.
zipcodeCounts["index_right"] = zipcodeCounts["index_right"].notna()
zipcodeCounts["count"] = np.where(zipcodeCounts["index_right"] == False, 0, 1)
zipcodeCounts = zipcodes[["zipcode", "geometry"]].merge(zipcodeCounts[["zipcode", "count"]].groupby("zipcode").sum("count"), on="zipcode", how="left").sort_values(["count", "zipcode"], ascending=False)
zipcodeCounts.head(1)

,zipcode,geometry,count
5,92104,"POLYGON ((-13040385.531 3863123.91, -13040453....",3


Step 6. Use a <u>within</u> operation to find park(s) (<em>parkCandidates</em>) in the zipcode(s) (<em>zipcodeCounts</em>) with the most skateboarding incidents → <em>parkFinalists</em>

In [ ]:
# Identify parks that are more than 1 mile away (parkCandidates) and are within the max-incident zipcode.
maxCountZipcode = zipcodeCounts.loc[zipcodeCounts['count'].idxmax(), "zipcode"]  # get max-incident zipcode
parkFinalists = parkCandidates.sjoin(zipcodeCounts.query("zipcode == @maxCountZipcode"), how="left").dropna(subset=["index_right"])[["park_name", "address", "community", "geometry"]]
parkFinalists

,park_name,address,community,geometry
49,North Park Community Park & Recreation Center,"4044 Idaho St., 92104",GREATER NORTH PARK,"POLYGON ((-13039287.959 3862472.995, -13039287..."
120,32nd Street Mini-Park,"3145 32ND ST., 92104",GREATER NORTH PARK,"POLYGON ((-13038308.19 3860596.963, -13038308...."
126,North Park Mini-Park,"2874 N PARK WAY, 92104",NORTH PARK,"POLYGON ((-13039047.957 3861834.037, -13038976..."
